In [1]:
# Import Depedencies 
import pandas as pd 

In [2]:
# Import cleaned csv
csvfile1 = "Resources/Cleaned_Vaccine_Case_Data.csv"

# Add as DataFrame
main_df = pd.read_csv(csvfile1)
main_df

,Unnamed: 0,POSTCODE,Total Population,Total Case Count,% Cases Over Age 18,% Cases Over Age 65,Total Cases Category,Total Vaccinated,Total Vaccinated Category,Median Household Income
0,0,85003,8499,1534,46.20,74.49,Greater Than 1000,4898,"1,001-5,000",44140
1,1,85004,7164,969,63.68,61.45,501-1000,4281,"1,001-5,000",41951
2,2,85006,27206,3272,35.85,54.55,Greater Than 1000,6780,"5,001-10,000",38121
3,3,85007,14428,2424,41.44,63.91,Greater Than 1000,4719,"1,001-5,000",34667
4,4,85008,62340,9079,33.01,56.27,Greater Than 1000,14187,"Greater Than 10,000",43574
...,...,...,...,...,...,...,...,...,...,...
339,360,86444,1254,60,34.27,49.25,11-100,450,"11-1,000",46389
340,361,86445,254,12,12.73,20.45,11-100,42,"11-1,000",51150
341,362,86502,1095,497,34.12,47.10,101-500,265,"11-1,000",18542
342,364,86505,8036,137,12.40,36.62,101-500,54,"11-1,000",23767
